In [106]:
# 先安裝line bot需要的套件 
# !pip install "line_bot_sdk==1.16.0"
# print('end')

In [107]:
# basic
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) #不要科學記號
pd.set_option('display.float_format',lambda x : '%.4f' % x)
pd.set_option('display.max_rows', 10)

# 獲得股票資訊
import pandas_datareader as pdr
import yfinance as yf

# 繪圖用的套件
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#獲得時間
import datetime as datetime
import time

#talib計算金融技術指標
import talib
from talib import abstract

import requests

today = datetime.datetime.now() 
day_ago = today - datetime.timedelta(days=100)

import pandas as pd
from datetime import date
import pymssql

# linebot
from linebot import ( LineBotApi, WebhookHandler )
from linebot.exceptions import ( InvalidSignatureError )
from linebot.models import *

print('...End')

...End


# 連接資料庫

In [108]:
conn = pymssql.connect(host='Database Host',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList
sql = 'select * from StrategyList'
df_stg = pd.read_sql(sql, conn)      # 2 StrategyList
sql = 'select * from TodaySignal'
df_signal = pd.read_sql(sql, conn)   # 3 TodaySignal

In [119]:
df_user

,ID,Name,0,1,2
0,u3f83uhd,Emery,0,1,0
1,bsbw4gwe,Sandy,1,0,1
2,svadvvw,Kevin,1,0,1
3,g43egsde,Ken,1,0,1
4,vsddsvsx,Amy,1,0,1
5,U53ba0f86ac04064b808aa924e057947b,Andy,1,0,1
6,fsavsv,Jack,1,0,1


In [120]:
stg_name = df_stg['StrategyName'].tolist()
stg_name = [str(x.strip()) for x in stg_name]  #去除空白!!
stg_par = df_stg['Parameter'].tolist() 
stg_par = [str(x.strip()) for x in stg_par]
stg_index = [str(x) for x in df_stg.index.tolist()]
print(stg_name,stg_par)

['BBAND', 'RSI', 'MACD'] ['BBANDS(df,timeperiod=20,nbdevup=1.0,nbdevdn=1.0,matype=0)', 'RSI(df,timeperiod=14)', 'MACD(df,fastperiod=12,slowperiod=26,signalperiod=9)']


# 個別使用者的推薦清單

In [121]:
# 先更新使用者的關注清單
# 將所有使用者關注策略存成list --> all_users_stg
def renew_user():
    global all_users_stg
    all_users_stg = []
    for user_index in range(len(df_user)):          #第0~6位
        one_user = []
        for i in stg_index:                         #[0,1,2]
            if df_user[i].loc[user_index] == 1: 
                one_user.append(i)
                print( '第',user_index, '個使用者，', stg_name[ int(i) ], '是否用:',df_user[i].loc[user_index])
        all_users_stg.append(one_user)

In [122]:
renew_user()
all_users_stg

第 0 個使用者， RSI 是否用: 1
第 1 個使用者， BBAND 是否用: 1
第 1 個使用者， MACD 是否用: 1
第 2 個使用者， BBAND 是否用: 1
第 2 個使用者， MACD 是否用: 1
第 3 個使用者， BBAND 是否用: 1
第 3 個使用者， MACD 是否用: 1
第 4 個使用者， BBAND 是否用: 1
第 4 個使用者， MACD 是否用: 1
第 5 個使用者， BBAND 是否用: 1
第 5 個使用者， MACD 是否用: 1
第 6 個使用者， BBAND 是否用: 1
第 6 個使用者， MACD 是否用: 1


[['1'], ['0', '2'], ['0', '2'], ['0', '2'], ['0', '2'], ['0', '2'], ['0', '2']]

In [123]:
# 測試 - 某位使用者
df_result = pd.DataFrame()
user_stg = all_users_stg[3]  #某位使用者
for i in [0,1,2]:                                  #符合數量前三
    if len(user_stg) - i > 0 :
        first = df_signal.T.loc[ user_stg ].sum() == len(user_stg) - i   
        df_first = df_signal[first].reset_index()                         #找出符合(user_stg-i)個策略 的標的，並重置index
        df_first.drop(['index'], axis = 1, inplace = True)
        #加上符合的數量
        df_first = pd.concat( [ df_first, pd.DataFrame([len(user_stg) - i] * len(df_first), columns = ['符合數量'])], axis = 1) 
        df_result = pd.concat([ df_result, df_first], axis = 0 )          #合併
        print(len(user_stg) - i, user_stg)

df_result.reset_index(inplace = True)
df_result.drop(['index'], axis = 1, inplace = True)
df_result

2 ['0', '2']
1 ['0', '2']


,StockSymbol,StockName,IndustryType,0,1,2,符合數量
0,2424,隴華,電腦及週邊設備業,1,0,1,2
1,8101,華冠,通信網路業,1,0,1,2
2,1471,首利,電子零組件業,1,0,0,1
3,1512,瑞利,汽車工業,1,0,0,1
4,2015,豐興,鋼鐵工業,0,0,1,1
...,...,...,...,...,...,...,...
15,3130,一零四,資訊服務業,0,0,1,1
16,3701,大眾控,電腦及週邊設備業,1,0,0,1
17,4536,拓凱,其他業,0,0,1,1
18,6625,必應,其他業,1,0,0,1


# Line Bot 傳訊息

In [127]:
def Daily_push():
    global all_users_stg
    # 主程式
    
    for user_index in range(len(df_user)):                   # 迴圈每位使用者
        # 某位使用者
        df_result = pd.DataFrame()
        user_stg = all_users_stg[user_index]
        for i in range(len(df_signal.columns) - 3):                               #符合數量前三
              if len(user_stg) - i > 0 :
                first = df_signal.T.loc[ user_stg ].sum() == len(user_stg) - i   
                df_first = df_signal[first].reset_index()                         #找出符合(user_stg-i)個策略 的標的，並重置index
                df_first.drop(['index'], axis = 1, inplace = True)
                #加上符合的數量
                df_first = pd.concat( [ df_first, pd.DataFrame([len(user_stg) - i] * len(df_first), columns = ['符合數量'])], axis = 1) 
                df_result = pd.concat([ df_result, df_first], axis = 0 )          #合併

        df_result.reset_index(inplace = True)
        df_result.drop(['index'], axis = 1, inplace = True)

        a_str = ''
        for stg_i in range(len(all_users_stg[user_index])):
            a_str = a_str + ' ' + stg_name[int(all_users_stg[user_index][stg_i])]

        # 建構清單 flex的格式 
        stock_list = []

        for i in range(len(df_result)):                        # 迴圈每檔股票
            column_list = []
            if i >= 150:
                break
            for c in ['StockSymbol','StockName','符合數量']:         # 迴圈一檔股票的每個欄位
                column_list.append({
                    "type": "text",
                    "text": str(df_result[c].loc[i]),
                    "contents": []
                  })        
            stock_list.append({
                "type": "box",
                "layout": "horizontal",
                "contents": column_list
              })

        # 一位使用者的 Flex 訊息
        contents = {
          "type": "bubble",
          "direction": "ltr",
          "footer": {
            "type": "box",
            "layout": "vertical",
            "contents": [
              {
                "type": "box",
                "layout": "vertical",
                "backgroundColor": "#F3B8B8FF",
                "contents": [
                  {
                    "type": "text",
                    "text": df_user['Name'].loc[user_index]+"的股票關注清單",
                    "color": "#000000FF",
                    "align": "center",
                    "contents": []
                  }
                ]
              },
              {
                "type": "text",
                "text": "關注策略："+a_str,
                "contents": []
              },
              {
                "type": "box",
                "layout": "horizontal",
                "backgroundColor": "#FCE5C0FF",
                "contents": [
                  {
                    "type": "text",
                    "text": "股票代號",
                    "contents": []
                  },
                  {
                    "type": "text",
                    "text": "名稱",
                    "contents": []
                  },
                  {
                    "type": "text",
                    "text": "符合數量",
                    "contents": []
                  }
                ]
              },        
            ] + stock_list
          }
        }

        message = FlexSendMessage(alt_text="hello", contents=contents)
        try:
            line_bot_api.push_message( df_user['ID'].loc[user_index], message )   
        except:
            print( '錯誤的使用者ID:', df_user['ID'].loc[user_index])
            pass

In [128]:
conn = pymssql.connect(host='Database Host',user='sa',password='linebot',database='QuantLineBot',charset='utf8')
cursor = conn.cursor()
sql = 'select * from UserList'
df_user = pd.read_sql(sql, conn)     # 1 UserList

In [129]:
# Channel Access Token
line_bot_api = LineBotApi('Line Bot 的 Channel access token ')
Daily_push()

錯誤的使用者ID: u3f83uhd                                
錯誤的使用者ID: bsbw4gwe                                
錯誤的使用者ID: svadvvw                                 
錯誤的使用者ID: g43egsde                                
錯誤的使用者ID: vsddsvsx                                
錯誤的使用者ID: fsavsv                                  


# 各式訊息type